In [28]:
import torch
import os
import pandas as pd

from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

from torchvision  import transforms

class Data(Dataset):
    def __init__(self, data, labels, transforms=None):
        self.data = data
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):

        image = self.data[idx]
        label = self.labels[idx]
        
        if self.transforms:
            image = self.transforms(image)

        return image, label 

batch_size = 64


preprocess = transforms.Compose([
    transforms.ConvertImageDtype(torch.float),  
    transforms.Resize((224, 224), antialias=True),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


train_data = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/train_data.pt")
train_labels = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/train_labels.pt")

val_data = train_data[:10000]
val_labels = train_labels[:10000]

train_data = train_data[10000:]
train_labels = train_labels[10000:]


train_dataset = Data(train_data, train_labels, transforms=preprocess)
val_dataset = Data(val_data, val_labels, transforms=preprocess)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)


test_data = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/test_data.pt")
test_labels = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/test_labels.pt")

test_dataset = Data(test_data, test_labels, transforms=preprocess)

test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


/tmp/ipykernel_508446/543658609.py:39: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_data = torch.load("/home/walke/college/cv/ass2/CV Assignment 2/Q1/train_data.pt")


In [29]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

model = models.resnet18()
model.fc = torch.nn.Linear(model.fc.in_features, 10)

model.to(device)

Using cuda device


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [30]:
from tqdm import tqdm

def train_loop(dataloader, model, loss_fn, optimizer):
    total = len(dataloader)
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0

    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in tqdm(enumerate(dataloader),desc="train", total = total):
        # Compute prediction and loss
        optimizer.zero_grad()
        
        X, y = X.to(device).float(), y.to(device)
        pred = model(X)
        loss = loss_fn(pred, y)
        train_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        

        correct += (pred.argmax(1) == y).type(torch.float).sum().item()


        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)


    train_loss /= num_batches
    correct /= size
    print(f"Training Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Training loss: {train_loss:>8f} \n")
    



def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    # model.load_state_dict(torch.load(model_path))

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            
            # _, y = torch.argmax(y, dim=1)  
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


def val_loop(dataloader, model, loss_fn):

    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device).float(), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()
            
            # _, y = torch.argmax(y, dim=1)  
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    val_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg Validation loss: {val_loss:>8f} \n")

    return correct

In [32]:

learning_rate = 1e-3
batch_size = 64
epochs = 5

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# model.to(device)
val_accuracy_highest = 0
val_accuracy_curr = 0

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    val_accuracy_curr = val_loop(val_dataloader, model, loss_fn)
    
    # if val_accuracy_curr > val_accuracy_highest:
    #     val_accuracy_highest = val_accuracy_curr
    #     model_path = 'model_1'
    #     print(f"Saving Model at epoch: {t+1}\n")
    #     torch.save(model.state_dict(), model_path)

    
print("Done!")



test_loop(test_dataloader, model, loss_fn)

Epoch 1
-------------------------------


train: 100%|██████████| 625/625 [02:00<00:00,  5.20it/s]


Training Error: 
 Accuracy: 65.8%, Avg Training loss: 0.966158 

Validation Error: 
 Accuracy: 69.4%, Avg Validation loss: 0.864021 

Epoch 2
-------------------------------


train: 100%|██████████| 625/625 [02:03<00:00,  5.05it/s]


Training Error: 
 Accuracy: 75.0%, Avg Training loss: 0.713186 

Validation Error: 
 Accuracy: 75.0%, Avg Validation loss: 0.714289 

Epoch 3
-------------------------------


...

Training Error: 
 Accuracy: 80.0%, Avg Training loss: 0.572156 

Validation Error: 
 Accuracy: 75.0%, Avg Validation loss: 0.716711 

Epoch 4
-------------------------------


train: 100%|██████████| 625/625 [02:04<00:00,  5.01it/s]


Training Error: 
 Accuracy: 83.8%, Avg Training loss: 0.465404 

Validation Error: 
 Accuracy: 79.4%, Avg Validation loss: 0.601660 

Epoch 5
-------------------------------


train: 100%|██████████| 625/625 [02:05<00:00,  4.99it/s]


Training Error: 
 Accuracy: 87.1%, Avg Training loss: 0.366606 

Validation Error: 
 Accuracy: 77.5%, Avg Validation loss: 0.718810 

Done!
Test Error: 
 Accuracy: 77.1%, Avg loss: 0.745082 

